# Занятие 4. Видео поиск (2)

In [2]:
import pathlib
import yaml
ROOT = pathlib.Path().resolve().parent.parent

with open(ROOT / "config.yml", "r") as f:
    config = yaml.safe_load(f)

In [3]:
# 3
DATA_PATH = ROOT / config['api']['DATA_PATH'] / 'seminar4'

query = "корейская косметика"
type_query = "video"

# Для поиска видео с учетом даты:
publishedAfter = '2020-01-01T00:00:00Z'
publishedBefore = '2022-01-01T00:00:00Z'

In [29]:
# 3.1
class KeyStorage:
    """
    Class to use in iteration
    """

    def __init__(self, key_arr: list[str]):
        self.key_gen = iter(key_arr)

    def get_next_key(self):
        """
        Returns API key to use
        :return:
        """
        return next(self.key_gen)

key_storage = KeyStorage(config['api']['keys'])

In [53]:
# 3.2 Просмотр API ключа
API_KEY = key_storage.get_next_key()

In [54]:
#4
import googleapiclient.discovery as api

In [55]:
#5
api_service_name = "youtube"
api_version = "v3"
    
youtube = api.build(api_service_name,
                        api_version, 
                       developerKey = API_KEY)

In [35]:
#8
import pandas as pd

In [36]:
# 18.1
# Первый заход БЕЗ аргумента order для видео
request = youtube.search().list(
        part ="snippet",
        maxResults=50,
        q = query,
        regionCode ="RU",
        type = type_query
)
response = request.execute()  
   
df_supplemented = pd.json_normalize(response['items'])
   
    # Цикл для прохода по всем следующим страницам с выдачей:
i = 0
while 'nextPageToken' in response.keys():
    request = youtube.search().list(
            part ="snippet",
            maxResults=50,
            q = query,
            type = type_query,
            pageToken = response['nextPageToken']
        )
    response = request.execute()  

    print(f'Итерация БЕЗ включения аргумента order №{i}')
    i += 1
        
    df_additional = pd.json_normalize(response['items'])
    df_supplemented = pd.concat([df_supplemented, df_additional])

    

print(f"Искомых объектов {response['pageInfo']['totalResults']}", \
      f"а найденных БЕЗ включения каких-либо значений аргумента order {len(df_supplemented.drop_duplicates(f'id.{type_query}Id'))}")

Итерация БЕЗ включения аргумента order №0
Итерация БЕЗ включения аргумента order №1
Итерация БЕЗ включения аргумента order №2
Итерация БЕЗ включения аргумента order №3
Итерация БЕЗ включения аргумента order №4
Итерация БЕЗ включения аргумента order №5
Итерация БЕЗ включения аргумента order №6
Итерация БЕЗ включения аргумента order №7
Итерация БЕЗ включения аргумента order №8
Итерация БЕЗ включения аргумента order №9
Итерация БЕЗ включения аргумента order №10
Искомых объектов 947437 а найденных БЕЗ включения каких-либо значений аргумента order 536


In [37]:
len(df_supplemented)

599

In [38]:
# Значения аргумента order:
order_list = ['date', 'rating', 'title', 'videoCount', 'viewCount']
order_list

['date', 'rating', 'title', 'videoCount', 'viewCount']

In [39]:
# 18.2
# Цикл для прохода по значениям аргумента order, внутри которых проход по всем страницам выдачи:
i = 0
for order in order_list:
    # Для остановки алгоритма, если все искомые объекты найдены
    # БЕЗ включения каких-либо значений аргумента order (в т.ч. вообще БЕЗ них):
    if len(df_supplemented.drop_duplicates(f'id.{type_query}Id')) < response['pageInfo']['totalResults']:
        
        # Первый заход с каждым значение аргумента order:
        request = youtube.search().list(
            part ="snippet",
            maxResults=50,
            q = query,
            type = type_query,
            order = order
        )
        response = request.execute()
    
        # Визуализация процесса:
        print(f'Итерация №{i}, "order" {order}, "items" {len(response["items"])}')
        i += 1
    
        # Занесение собранных данных в итоговую таблицу df_supplemented:
        df = pd.json_normalize(response["items"])
        df_supplemented = pd.concat([df, df_supplemented])
        
        # Заходы с тем же значением аргумента order на следующие страницы:
        while ('nextPageToken' in response.keys())\
        & (len(df_supplemented.drop_duplicates(f'id.{type_query}Id')) < response['pageInfo']['totalResults'])\
        & (len(response["items"]) > 0):
        # второе условие -- для остановки алгоритма, если все искомые объекты найдены
        # БЕЗ какой-то из следующих страниц ( в т.ч. вообще БЕЗ них)
        # третье условие -- для остановки алгоритма, если предыдущая страница выдачи содержит 0 объектов    
        
            request = youtube.search().list(
                part ="snippet",
                maxResults=50,
                q = query,
                type = type_query,
                pageToken = response['nextPageToken'],
                order = order
            )
            response = request.execute()  
    
            # Визуализация процесса:
            print(f'Итерация №{i}, "order" {order}, "items" {len(response["items"])}')
            
            i += 1
        
            # Занесение собранных данных в итоговую таблицу df_supplemented:
            df_additional = pd.json_normalize(response["items"])
            df_supplemented = pd.concat([df_supplemented, df_additional])
    
        # Сохранение составленных для каждого значения аргумента order таблиц в Excel -- на всякий случай:
        df_supplemented.to_excel(DATA_PATH / f'{type_query.capitalize()}_Sorted_by_{order}.xlsx')
    else:
        print('Все искомые объекты найдены БЕЗ включения некоторых значений аргумента order (в т.ч. вообще БЕЗ них)')

Итерация №0, "order" date, "items" 50
Итерация №1, "order" date, "items" 50
Итерация №2, "order" date, "items" 50
Итерация №3, "order" date, "items" 50
Итерация №4, "order" date, "items" 50
Итерация №5, "order" date, "items" 50
Итерация №6, "order" date, "items" 50
Итерация №7, "order" date, "items" 50
Итерация №8, "order" date, "items" 50
Итерация №9, "order" date, "items" 50
Итерация №10, "order" date, "items" 50
Итерация №11, "order" date, "items" 17
Итерация №12, "order" rating, "items" 50
Итерация №13, "order" rating, "items" 50
Итерация №14, "order" rating, "items" 50
Итерация №15, "order" rating, "items" 50
Итерация №16, "order" rating, "items" 50
Итерация №17, "order" rating, "items" 50
Итерация №18, "order" rating, "items" 50
Итерация №19, "order" rating, "items" 50
Итерация №20, "order" rating, "items" 50
Итерация №21, "order" rating, "items" 50
Итерация №22, "order" rating, "items" 50
Итерация №23, "order" rating, "items" 18
Итерация №24, "order" title, "items" 50
Итерация №

In [42]:
# 19
# Удаление дубликатов каналов (по их ID) и запись в Excel:
df_supplemented.index = range(1,len(df_supplemented)+1) # сквозной индекс для итоговой таблицы
df_supplemented = df_supplemented.drop_duplicates()
display(df_supplemented)


,kind,etag,id.kind,id.videoId,snippet.publishedAt,snippet.channelId,snippet.title,snippet.description,snippet.thumbnails.default.url,snippet.thumbnails.default.width,snippet.thumbnails.default.height,snippet.thumbnails.medium.url,snippet.thumbnails.medium.width,snippet.thumbnails.medium.height,snippet.thumbnails.high.url,snippet.thumbnails.high.width,snippet.thumbnails.high.height,snippet.channelTitle,snippet.liveBroadcastContent,snippet.publishTime
1,youtube#searchResult,EQDFH4sq29F4tLopSvBzT1Owm1M,youtube#video,AAjsSrjZXsw,2021-03-10T13:46:47Z,UCh29SdTOPdjBb-n3Vop1lYg,Расследование | Паленая косметика на Ozon и Wi...,На российских маркетплейсах продается огромное...,https://i.ytimg.com/vi/AAjsSrjZXsw/default.jpg,120,90,https://i.ytimg.com/vi/AAjsSrjZXsw/mqdefault.jpg,320,180,https://i.ytimg.com/vi/AAjsSrjZXsw/hqdefault.jpg,480,360,katyakonasova,none,2021-03-10T13:46:47Z
2,youtube#searchResult,P1X20DCj2WrucSb9ldNXa1bYGQU,youtube#video,yk4Wjr3j7QY,2023-02-05T04:32:39Z,UCfcMgRNXey6TQkov9mgVgoA,Пляжная мода в Корее #этокореядетка #корейская...,,https://i.ytimg.com/vi/yk4Wjr3j7QY/default.jpg,120,90,https://i.ytimg.com/vi/yk4Wjr3j7QY/mqdefault.jpg,320,180,https://i.ytimg.com/vi/yk4Wjr3j7QY/hqdefault.jpg,480,360,altyshkorea,none,2023-02-05T04:32:39Z
3,youtube#searchResult,KC_LG8xTt9DjwtELdwXsHef8Y2g,youtube#video,Wi49rKRM85U,2022-06-20T12:57:48Z,UCphnZS52v_lkv5RSl5n70tQ,한국인들은 절대 안사는 화장품 КОСМЕТИКА КОТОРАЯ КОРЕЯНКИ ВО...,подписаться: https://goo.gl/3QbeUf -----------...,https://i.ytimg.com/vi/Wi49rKRM85U/default.jpg,120,90,https://i.ytimg.com/vi/Wi49rKRM85U/mqdefault.jpg,320,180,https://i.ytimg.com/vi/Wi49rKRM85U/hqdefault.jpg,480,360,Kyunghamin твоя корейская онни,none,2022-06-20T12:57:48Z
4,youtube#searchResult,jLZGccW4KN5uL3WAMK7JggpPUsc,youtube#video,kSiaxefvCgM,2023-02-11T12:43:25Z,UCfcMgRNXey6TQkov9mgVgoA,Жилищный вопрос: Корея #дорамы #корейскийязык ...,,https://i.ytimg.com/vi/kSiaxefvCgM/default.jpg,120,90,https://i.ytimg.com/vi/kSiaxefvCgM/mqdefault.jpg,320,180,https://i.ytimg.com/vi/kSiaxefvCgM/hqdefault.jpg,480,360,altyshkorea,none,2023-02-11T12:43:25Z
5,youtube#searchResult,dltiYtp7e0SGjSrmXCopbsnWyVQ,youtube#video,4nqfUzIz7kQ,2021-11-12T14:59:32Z,UCphnZS52v_lkv5RSl5n70tQ,Кореянка в Золотом Яблоке. Почему русские поку...,кореянка#золотоеяблоко#корейскаякосметика прив...,https://i.ytimg.com/vi/4nqfUzIz7kQ/default.jpg,120,90,https://i.ytimg.com/vi/4nqfUzIz7kQ/mqdefault.jpg,320,180,https://i.ytimg.com/vi/4nqfUzIz7kQ/hqdefault.jpg,480,360,Kyunghamin твоя корейская онни,none,2021-11-12T14:59:32Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
823,youtube#searchResult,mBOUDV2CcPUuRmwdRamWekcdljo,youtube#video,h2B_zgdX__Y,2019-02-23T18:27:23Z,UCaqikEbNbRJaUv-DKPFPOHQ,Корейская косметика#крем#farm stay#зеленый чай,Питательный корейский крем от Farm stay . Подр...,https://i.ytimg.com/vi/h2B_zgdX__Y/default.jpg,120,90,https://i.ytimg.com/vi/h2B_zgdX__Y/mqdefault.jpg,320,180,https://i.ytimg.com/vi/h2B_zgdX__Y/hqdefault.jpg,480,360,Корейская косметика,none,2019-02-23T18:27:23Z
824,youtube#searchResult,kd7-kiciAX96a0us39wBpxi_hI0,youtube#video,VlWKjch_m5c,2020-12-05T12:11:49Z,UCaRfOL6s_IPjQ3LKcMqDxZg,Самые лучшие шампуни для жирной кожи головы | ...,Шампунь против перхоти La'Dor Anti-Dandruff Sh...,https://i.ytimg.com/vi/VlWKjch_m5c/default.jpg,120,90,https://i.ytimg.com/vi/VlWKjch_m5c/mqdefault.jpg,320,180,https://i.ytimg.com/vi/VlWKjch_m5c/hqdefault.jpg,480,360,Lotus Fresh Корейская Косметика,none,2020-12-05T12:11:49Z
825,youtube#searchResult,tHOYnpvWStjOBH1OdVwpJCNOJMc,youtube#video,AZj6WbmSH7c,2022-06-28T09:01:32Z,UCYfIVot-gXDXDnPa6oBur3w,Обзор на набор Atomy Absolute CellActive. Прем...,Всем привет! В этом видео я сделала обзор на н...,https://i.ytimg.com/vi/AZj6WbmSH7c/default.jpg,120,90,https://i.ytimg.com/vi/AZj6WbmSH7c/mqdefault.jpg,320,180,https://i.ytimg.com/vi/AZj6WbmSH7c/hqdefault.jpg,480,360,Кореянка о косметике,none,2022-06-28T09:01:32Z
826,youtube#searchResult,3_OQBRon3OK8B7SJjgP_Fq1

In [43]:
# 20

df_supplemented.to_excel(DATA_PATH /f'{query}_{type_query.capitalize()}_Not_sorted+Sorted.xlsx', index=False)

In [44]:
df_supplemented = pd.read_excel(DATA_PATH / f'{query}_{type_query.capitalize()}_Not_sorted+Sorted.xlsx')
# возврат к выдаче без учета date; указание, что индекс в первом столбце
df_supplemented

,kind,etag,id.kind,id.videoId,snippet.publishedAt,snippet.channelId,snippet.title,snippet.description,snippet.thumbnails.default.url,snippet.thumbnails.default.width,snippet.thumbnails.default.height,snippet.thumbnails.medium.url,snippet.thumbnails.medium.width,snippet.thumbnails.medium.height,snippet.thumbnails.high.url,snippet.thumbnails.high.width,snippet.thumbnails.high.height,snippet.channelTitle,snippet.liveBroadcastContent,snippet.publishTime
0,youtube#searchResult,EQDFH4sq29F4tLopSvBzT1Owm1M,youtube#video,AAjsSrjZXsw,2021-03-10T13:46:47Z,UCh29SdTOPdjBb-n3Vop1lYg,Расследование | Паленая косметика на Ozon и Wi...,На российских маркетплейсах продается огромное...,https://i.ytimg.com/vi/AAjsSrjZXsw/default.jpg,120,90,https://i.ytimg.com/vi/AAjsSrjZXsw/mqdefault.jpg,320,180,https://i.ytimg.com/vi/AAjsSrjZXsw/hqdefault.jpg,480,360,katyakonasova,none,2021-03-10T13:46:47Z
1,youtube#searchResult,P1X20DCj2WrucSb9ldNXa1bYGQU,youtube#video,yk4Wjr3j7QY,2023-02-05T04:32:39Z,UCfcMgRNXey6TQkov9mgVgoA,Пляжная мода в Корее #этокореядетка #корейская...,NaN,https://i.ytimg.com/vi/yk4Wjr3j7QY/default.jpg,120,90,https://i.ytimg.com/vi/yk4Wjr3j7QY/mqdefault.jpg,320,180,https://i.ytimg.com/vi/yk4Wjr3j7QY/hqdefault.jpg,480,360,altyshkorea,none,2023-02-05T04:32:39Z
2,youtube#searchResult,KC_LG8xTt9DjwtELdwXsHef8Y2g,youtube#video,Wi49rKRM85U,2022-06-20T12:57:48Z,UCphnZS52v_lkv5RSl5n70tQ,한국인들은 절대 안사는 화장품 КОСМЕТИКА КОТОРАЯ КОРЕЯНКИ ВО...,подписаться: https://goo.gl/3QbeUf -----------...,https://i.ytimg.com/vi/Wi49rKRM85U/default.jpg,120,90,https://i.ytimg.com/vi/Wi49rKRM85U/mqdefault.jpg,320,180,https://i.ytimg.com/vi/Wi49rKRM85U/hqdefault.jpg,480,360,Kyunghamin твоя корейская онни,none,2022-06-20T12:57:48Z
3,youtube#searchResult,jLZGccW4KN5uL3WAMK7JggpPUsc,youtube#video,kSiaxefvCgM,2023-02-11T12:43:25Z,UCfcMgRNXey6TQkov9mgVgoA,Жилищный вопрос: Корея #дорамы #корейскийязык ...,NaN,https://i.ytimg.com/vi/kSiaxefvCgM/default.jpg,120,90,https://i.ytimg.com/vi/kSiaxefvCgM/mqdefault.jpg,320,180,https://i.ytimg.com/vi/kSiaxefvCgM/hqdefault.jpg,480,360,altyshkorea,none,2023-02-11T12:43:25Z
4,youtube#searchResult,dltiYtp7e0SGjSrmXCopbsnWyVQ,youtube#video,4nqfUzIz7kQ,2021-11-12T14:59:32Z,UCphnZS52v_lkv5RSl5n70tQ,Кореянка в Золотом Яблоке. Почему русские поку...,кореянка#золотоеяблоко#корейскаякосметика прив...,https://i.ytimg.com/vi/4nqfUzIz7kQ/default.jpg,120,90,https://i.ytimg.com/vi/4nqfUzIz7kQ/mqdefault.jpg,320,180,https://i.ytimg.com/vi/4nqfUzIz7kQ/hqdefault.jpg,480,360,Kyunghamin твоя корейская онни,none,2021-11-12T14:59:32Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
822,youtube#searchResult,mBOUDV2CcPUuRmwdRamWekcdljo,youtube#video,h2B_zgdX__Y,2019-02-23T18:27:23Z,UCaqikEbNbRJaUv-DKPFPOHQ,Корейская косметика#крем#farm stay#зеленый чай,Питательный корейский крем от Farm stay . Подр...,https://i.ytimg.com/vi/h2B_zgdX__Y/default.jpg,120,90,https://i.ytimg.com/vi/h2B_zgdX__Y/mqdefault.jpg,320,180,https://i.ytimg.com/vi/h2B_zgdX__Y/hqdefault.jpg,480,360,Корейская косметика,none,2019-02-23T18:27:23Z
823,youtube#searchResult,kd7-kiciAX96a0us39wBpxi_hI0,youtube#video,VlWKjch_m5c,2020-12-05T12:11:49Z,UCaRfOL6s_IPjQ3LKcMqDxZg,Самые лучшие шампуни для жирной кожи головы | ...,Шампунь против перхоти La'Dor Anti-Dandruff Sh...,https://i.ytimg.com/vi/VlWKjch_m5c/default.jpg,120,90,https://i.ytimg.com/vi/VlWKjch_m5c/mqdefault.jpg,320,180,https://i.ytimg.com/vi/VlWKjch_m5c/hqdefault.jpg,480,360,Lotus Fresh Корейская Косметика,none,2020-12-05T12:11:49Z
824,youtube#searchResult,tHOYnpvWStjOBH1OdVwpJCNOJMc,youtube#video,AZj6WbmSH7c,2022-06-28T09:01:32Z,UCYfIVot-gXDXDnPa6oBur3w,Обзор на набор Atomy Absolute CellActive. Прем...,Всем привет! В этом видео я сделала обзор на н...,https://i.ytimg.com/vi/AZj6WbmSH7c/default.jpg,120,90,https://i.ytimg.com/vi/AZj6WbmSH7c/mqdefault.jpg,320,180,https://i.ytimg.com/vi/AZj6WbmSH7c/hqdefault.jpg,480,360,Кореянка о косметике,none,2022-06-28T09:01:32Z
825,youtube#searchResult,3_OQBRon3OK8B7SJj

In [45]:
#19.1 Запросы БЕЗ включения аргумента order, но с учетом даты
# Первый заход:
request = youtube.search().list(
        part ="snippet",
        maxResults=50,
        q = query,
        type = type_query,
        publishedAfter=publishedAfter,
        publishedBefore=publishedBefore
)
response = request.execute()  
   
df_supplemented = pd.json_normalize(response['items'])
   
    # Цикл для прохода по всем следующим страницам с выдачей:
i = 0
while 'nextPageToken' in response.keys():
    request = youtube.search().list(
            part ="snippet",
            maxResults=50,
            q = query,
            type = type_query,
            pageToken = response['nextPageToken'],
            publishedAfter=publishedAfter,
            publishedBefore=publishedBefore
    )
    response = request.execute()  
    
    print(f'Итерация №{i}, "date" {publishedAfter[:11]} - {publishedBefore[:11]}')
    i += 1
        
    df_additional = pd.json_normalize(response['items'])
    df_supplemented = pd.concat([df_supplemented, df_additional])
    
    

print(f"Искомых объектов {response['pageInfo']['totalResults']}", \
      f"а найденных БЕЗ включения каких-либо значений аргумента order {len(df_supplemented.drop_duplicates(f'id.{type_query}Id'))}")

Итерация №0, "date" 2020-01-01T - 2022-01-01T
Итерация №1, "date" 2020-01-01T - 2022-01-01T
Итерация №2, "date" 2020-01-01T - 2022-01-01T
Итерация №3, "date" 2020-01-01T - 2022-01-01T
Итерация №4, "date" 2020-01-01T - 2022-01-01T
Итерация №5, "date" 2020-01-01T - 2022-01-01T
Итерация №6, "date" 2020-01-01T - 2022-01-01T
Итерация №7, "date" 2020-01-01T - 2022-01-01T
Итерация №8, "date" 2020-01-01T - 2022-01-01T
Итерация №9, "date" 2020-01-01T - 2022-01-01T
Искомых объектов 420428 а найденных БЕЗ включения каких-либо значений аргумента order 435


In [46]:
# 19.2
# Цикл для прохода по значениям аргумента order, внутри которых проход по всем страницам выдачи:
# Думаю, можно не выполнять, так как большинство включено в искомые объекты

In [47]:
# 21 Проверка совпадения двух столбцов из выдачи:
df_supplemented = pd.read_excel(DATA_PATH / f'{query}_{type_query.capitalize()}_Not_sorted+Sorted.xlsx', index_col=0)
# возврат к выдаче без учета date; указание, что индекс в первом столбце

display(
    df_supplemented,
    # df_supplemented['snippet.publishedAt'] == df_supplemented['snippet.publishTime'],
    f"Совпадений двух столбцов: {sum(df_supplemented['snippet.publishedAt'] == df_supplemented['snippet.publishTime'])}"
    )

,etag,id.kind,id.videoId,snippet.publishedAt,snippet.channelId,snippet.title,snippet.description,snippet.thumbnails.default.url,snippet.thumbnails.default.width,snippet.thumbnails.default.height,snippet.thumbnails.medium.url,snippet.thumbnails.medium.width,snippet.thumbnails.medium.height,snippet.thumbnails.high.url,snippet.thumbnails.high.width,snippet.thumbnails.high.height,snippet.channelTitle,snippet.liveBroadcastContent,snippet.publishTime
kind,,,,,,,,,,,,,,,,,,,
youtube#searchResult,EQDFH4sq29F4tLopSvBzT1Owm1M,youtube#video,AAjsSrjZXsw,2021-03-10T13:46:47Z,UCh29SdTOPdjBb-n3Vop1lYg,Расследование | Паленая косметика на Ozon и Wi...,На российских маркетплейсах продается огромное...,https://i.ytimg.com/vi/AAjsSrjZXsw/default.jpg,120,90,https://i.ytimg.com/vi/AAjsSrjZXsw/mqdefault.jpg,320,180,https://i.ytimg.com/vi/AAjsSrjZXsw/hqdefault.jpg,480,360,katyakonasova,none,2021-03-10T13:46:47Z
youtube#searchResult,P1X20DCj2WrucSb9ldNXa1bYGQU,youtube#video,yk4Wjr3j7QY,2023-02-05T04:32:39Z,UCfcMgRNXey6TQkov9mgVgoA,Пляжная мода в Корее #этокореядетка #корейская...,NaN,https://i.ytimg.com/vi/yk4Wjr3j7QY/default.jpg,120,90,https://i.ytimg.com/vi/yk4Wjr3j7QY/mqdefault.jpg,320,180,https://i.ytimg.com/vi/yk4Wjr3j7QY/hqdefault.jpg,480,360,altyshkorea,none,2023-02-05T04:32:39Z
youtube#searchResult,KC_LG8xTt9DjwtELdwXsHef8Y2g,youtube#video,Wi49rKRM85U,2022-06-20T12:57:48Z,UCphnZS52v_lkv5RSl5n70tQ,한국인들은 절대 안사는 화장품 КОСМЕТИКА КОТОРАЯ КОРЕЯНКИ ВО...,подписаться: https://goo.gl/3QbeUf -----------...,https://i.ytimg.com/vi/Wi49rKRM85U/default.jpg,120,90,https://i.ytimg.com/vi/Wi49rKRM85U/mqdefault.jpg,320,180,https://i.ytimg.com/vi/Wi49rKRM85U/hqdefault.jpg,480,360,Kyunghamin твоя корейская онни,none,2022-06-20T12:57:48Z
youtube#searchResult,jLZGccW4KN5uL3WAMK7JggpPUsc,youtube#video,kSiaxefvCgM,2023-02-11T12:43:25Z,UCfcMgRNXey6TQkov9mgVgoA,Жилищный вопрос: Корея #дорамы #корейскийязык ...,NaN,https://i.ytimg.com/vi/kSiaxefvCgM/default.jpg,120,90,https://i.ytimg.com/vi/kSiaxefvCgM/mqdefault.jpg,320,180,https://i.ytimg.com/vi/kSiaxefvCgM/hqdefault.jpg,480,360,altyshkorea,none,2023-02-11T12:43:25Z
youtube#searchResult,dltiYtp7e0SGjSrmXCopbsnWyVQ,youtube#video,4nqfUzIz7kQ,2021-11-12T14:59:32Z,UCphnZS52v_lkv5RSl5n70tQ,Кореянка в Золотом Яблоке. Почему русские поку...,кореянка#золотоеяблоко#корейскаякосметика прив...,https://i.ytimg.com/vi/4nqfUzIz7kQ/default.jpg,120,90,https://i.ytimg.com/vi/4nqfUzIz7kQ/mqdefault.jpg,320,180,https://i.ytimg.com/vi/4nqfUzIz7kQ/hqdefault.jpg,480,360,Kyunghamin твоя корейская онни,none,2021-11-12T14:59:32Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
youtube#searchResult,mBOUDV2CcPUuRmwdRamWekcdljo,youtube#video,h2B_zgdX__Y,2019-02-23T18:27:23Z,UCaqikEbNbRJaUv-DKPFPOHQ,Корейская косметика#крем#farm stay#зеленый чай,Питательный корейский крем от Farm stay . Подр...,https://i.ytimg.com/vi/h2B_zgdX__Y/default.jpg,120,90,https://i.ytimg.com/vi/h2B_zgdX__Y/mqdefault.jpg,320,180,https://i.ytimg.com/vi/h2B_zgdX__Y/hqdefault.jpg,480,360,Корейская косметика,none,2019-02-23T18:27:23Z
youtube#searchResult,kd7-kiciAX96a0us39wBpxi_hI0,youtube#video,VlWKjch_m5c,2020-12-05T12:11:49Z,UCaRfOL6s_IPjQ3LKcMqDxZg,Самые лучшие шампуни для жирной кожи головы | ...,Шампунь против перхоти La'Dor Anti-Dandruff Sh...,https://i.ytimg.com/vi/VlWKjch_m5c/default.jpg,120,90,https://i.ytimg.com/vi/VlWKjch_m5c/mqdefault.jpg,320,180,https://i.ytimg.com/vi/VlWKjch_m5c/hqdefault.jpg,480,360,Lotus Fresh Корейская Косметика,none,2020-12-05T12:11:49Z
youtube#searchResult,tHOYnpvWStjOBH1OdVwpJCNOJMc,youtube#video,AZj6WbmSH7c,2022-06-28T09:01:32Z,UCYfIVot-gXDXDnPa6oBur3w,Обзор на набор Atomy Absolute CellActive. Прем...,Всем привет! В этом видео я сделала обзор на н...,https://i.ytimg.com/vi/AZj6WbmSH7c/default.jpg,120,90,https://i.ytimg.com/vi/AZj6WbmSH7c/mqdefault.jpg,320,180,https://i.ytimg.com/vi/AZj6WbmSH7c/hqdefault.jpg,480,360,Кореянка о косметике,none,2022-06-28T09:01:32Z


'Совпадений двух столбцов: 827'

In [48]:
# 22 Проверка индексирования содержимого столбца snippet.publishedAt и строчки №1:
df_supplemented['snippet.publishedAt'][1][:4]

'2023'

In [59]:
# 23 Lambda-функция для перезаписи в столбец snippet.publishedAt только года:
df_supplemented['year'] = df_supplemented['snippet.publishedAt'].apply(lambda yyyy: int(yyyy[:4]))
df_supplemented['year']

TypeError: 'int' object is not subscriptable

In [50]:
# 24 Поиск самого раннего года:
min_date = min(df_supplemented['snippet.publishedAt'])
max_date = max(df_supplemented['snippet.publishedAt'])

print("Минимальный год с видео", min_date)
print("Максимальный год с видео", max_date)

Минимальный год с видео 2013
Максимальный год с видео 2023


In [56]:
#25.1 Запросы БЕЗ включения аргумента order, но с учетом даты

# На всякий случай получим следующий ключ

i = 0
year = 2018

while year <= max_date:
     
    # Первый заход С включением аргумента date:
    request = youtube.search().list(
        part ="snippet",
        maxResults=50,
        q = query,
        regionCode ="RU",
        type = type_query,
        publishedAfter=f'{year}-01-01T00:00:00Z'
    )
    response = request.execute()  
   
    df_additional = pd.json_normalize(response['items'])
    df_supplemented = pd.concat([df_supplemented, df_additional])
   
    # Цикл для прохода по всем следующим страницам с выдачей:
    while 'nextPageToken' in response.keys():
        request = youtube.search().list(
            part ="snippet",
            maxResults=50,
            q = query,
            regionCode ="RU",
            type = type_query,
            pageToken = response['nextPageToken'],
            publishedAfter=f'{year}-01-01T00:00:00Z'
        )
        response = request.execute()  
    
        # Визуализация процесса:
        print(f'Итерация №{i}, "date" {year}')
        i += 1
        
        df_additional = pd.json_normalize(response['items'])
        df_supplemented = pd.concat([df_supplemented, df_additional])
    

    print(f"Искомых объектов с {year} года {response['pageInfo']['totalResults']}, \
      а найденных {len(df_supplemented.drop_duplicates(f'id.{type_query}Id'))}")

    year += 1

Итерация №0, "date" 2018
Итерация №1, "date" 2018
Итерация №2, "date" 2018
Итерация №3, "date" 2018
Итерация №4, "date" 2018
Итерация №5, "date" 2018
Итерация №6, "date" 2018
Итерация №7, "date" 2018
Итерация №8, "date" 2018
Итерация №9, "date" 2018
Итерация №10, "date" 2018
Искомых объектов с 2018 года 830595,       а найденных 1007
Итерация №11, "date" 2019
Итерация №12, "date" 2019
Итерация №13, "date" 2019
Итерация №14, "date" 2019
Итерация №15, "date" 2019
Итерация №16, "date" 2019
Итерация №17, "date" 2019
Итерация №18, "date" 2019
Итерация №19, "date" 2019
Итерация №20, "date" 2019
Итерация №21, "date" 2019
Искомых объектов с 2019 года 776840,       а найденных 1031
Итерация №22, "date" 2020
Итерация №23, "date" 2020
Итерация №24, "date" 2020
Итерация №25, "date" 2020
Итерация №26, "date" 2020
Итерация №27, "date" 2020
Итерация №28, "date" 2020
Итерация №29, "date" 2020
Итерация №30, "date" 2020
Итерация №31, "date" 2020
Итерация №32, "date" 2020
Искомых объектов с 2020 года 592

In [58]:
# 20
# Удаление дубликатов каналов (по их ID) и запись в Excel:
df_supplemented.index = range(1,len(df_supplemented)+1) # сквозной индекс для итоговой таблицы
df_supplemented = df_supplemented.drop_duplicates(subset=['id.videoId'])
display(df_supplemented)


,etag,id.kind,id.videoId,snippet.publishedAt,snippet.channelId,snippet.title,snippet.description,snippet.thumbnails.default.url,snippet.thumbnails.default.width,snippet.thumbnails.default.height,snippet.thumbnails.medium.url,snippet.thumbnails.medium.width,snippet.thumbnails.medium.height,snippet.thumbnails.high.url,snippet.thumbnails.high.width,snippet.thumbnails.high.height,snippet.channelTitle,snippet.liveBroadcastContent,snippet.publishTime,kind
1,EQDFH4sq29F4tLopSvBzT1Owm1M,youtube#video,AAjsSrjZXsw,2021,UCh29SdTOPdjBb-n3Vop1lYg,Расследование | Паленая косметика на Ozon и Wi...,На российских маркетплейсах продается огромное...,https://i.ytimg.com/vi/AAjsSrjZXsw/default.jpg,120,90,https://i.ytimg.com/vi/AAjsSrjZXsw/mqdefault.jpg,320,180,https://i.ytimg.com/vi/AAjsSrjZXsw/hqdefault.jpg,480,360,katyakonasova,none,2021-03-10T13:46:47Z,NaN
2,P1X20DCj2WrucSb9ldNXa1bYGQU,youtube#video,yk4Wjr3j7QY,2023,UCfcMgRNXey6TQkov9mgVgoA,Пляжная мода в Корее #этокореядетка #корейская...,NaN,https://i.ytimg.com/vi/yk4Wjr3j7QY/default.jpg,120,90,https://i.ytimg.com/vi/yk4Wjr3j7QY/mqdefault.jpg,320,180,https://i.ytimg.com/vi/yk4Wjr3j7QY/hqdefault.jpg,480,360,altyshkorea,none,2023-02-05T04:32:39Z,NaN
3,KC_LG8xTt9DjwtELdwXsHef8Y2g,youtube#video,Wi49rKRM85U,2022,UCphnZS52v_lkv5RSl5n70tQ,한국인들은 절대 안사는 화장품 КОСМЕТИКА КОТОРАЯ КОРЕЯНКИ ВО...,подписаться: https://goo.gl/3QbeUf -----------...,https://i.ytimg.com/vi/Wi49rKRM85U/default.jpg,120,90,https://i.ytimg.com/vi/Wi49rKRM85U/mqdefault.jpg,320,180,https://i.ytimg.com/vi/Wi49rKRM85U/hqdefault.jpg,480,360,Kyunghamin твоя корейская онни,none,2022-06-20T12:57:48Z,NaN
4,jLZGccW4KN5uL3WAMK7JggpPUsc,youtube#video,kSiaxefvCgM,2023,UCfcMgRNXey6TQkov9mgVgoA,Жилищный вопрос: Корея #дорамы #корейскийязык ...,NaN,https://i.ytimg.com/vi/kSiaxefvCgM/default.jpg,120,90,https://i.ytimg.com/vi/kSiaxefvCgM/mqdefault.jpg,320,180,https://i.ytimg.com/vi/kSiaxefvCgM/hqdefault.jpg,480,360,altyshkorea,none,2023-02-11T12:43:25Z,NaN
5,dltiYtp7e0SGjSrmXCopbsnWyVQ,youtube#video,4nqfUzIz7kQ,2021,UCphnZS52v_lkv5RSl5n70tQ,Кореянка в Золотом Яблоке. Почему русские поку...,кореянка#золотоеяблоко#корейскаякосметика прив...,https://i.ytimg.com/vi/4nqfUzIz7kQ/default.jpg,120,90,https://i.ytimg.com/vi/4nqfUzIz7kQ/mqdefault.jpg,320,180,https://i.ytimg.com/vi/4nqfUzIz7kQ/hqdefault.jpg,480,360,Kyunghamin твоя корейская онни,none,2021-11-12T14:59:32Z,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1973,T5xiMJIvrxeWR1zTaA0zegMQxnM,youtube#video,SNRrXxyiwk0,2023-03-20T15:43:32Z,UC2wyUPnUdd_o2JYPIUuDgoA,#атоми #корейскаякосметика #атомибизнес #уходз...,,https://i.ytimg.com/vi/SNRrXxyiwk0/default.jpg,120,90,https://i.ytimg.com/vi/SNRrXxyiwk0/mqdefault.jpg,320,180,https://i.ytimg.com/vi/SNRrXxyiwk0/hqdefault.jpg,480,360,Ксения Харенская,none,2023-03-20T15:43:32Z,youtube#searchResult
1974,1B0Q8RCO3FKkXFfQr5-DYSM-z5M,youtube#video,NPGuTcFkdHw,2023-03-13T22:15:57Z,UCXdscmYnR_wgN0uT0C-ug8w,"Немного закулисья, когда работаю и консультиру...",,https://i.ytimg.com/vi/NPGuTcFkdHw/default.jpg,120,90,https://i.ytimg.com/vi/NPGuTcFkdHw/mqdefault.jpg,320,180,https://i.ytimg.com/vi/NPGuTcFkdHw/hqdefault.jpg,480,360,PRO. beauty-bar,none,2023-03-13T22:15:57Z,youtube#searchResult
1975,n-lXr1sktVRV6D35_-xOKaej5yU,youtube#video,oqFLCIng4Bo,2023-03-29T15:32:24Z,UC2wyUPnUdd_o2JYPIUuDgoA,#фриланс #работаудаленно #атомибизнес #атомикр...,,https://i.ytimg.com/vi/oqFLCIng4Bo/default.jpg,120,90,https://i.ytimg.com/vi/oqFLCIng4Bo/mqdefault.jpg,320,180,https://i.ytimg.com/vi/oqFLCIng4Bo/hqdefault.jpg,480,360,Ксения Харенская,none,2023-03-29T15:32:24Z,youtube#searchResult
1976,spiZ5extV7ieubjGA-UBsCNLSRY,youtube#video,6glsQ1LyRXM,2023-03-19T17:00:07Z,UCbkr6G0g-d2HH5p5QYPyfag,Toner | Luon #luon #корейскаякосметика,,https://i.ytimg.com/vi/6glsQ1LyRXM/default.jpg,120,90,https://i.ytimg.com/vi/6glsQ1LyRXM/mqdefault.jpg,320,180,https://i.ytimg.com/vi/6glsQ1LyRXM/hqdefault.jpg,480,360,LUON,none,2023-03-19T17:00:07Z,youtube#searchResult


In [60]:
import os

# сохранение файлов в специальной директории

if not os.path.exists(DATA_PATH):
    os.makedirs(DATA_PATH)

df_supplemented.to_excel(DATA_PATH / f'{query}_{type_query.capitalize()}_Not_sorted+Sorted+Date.xlsx')